### 🔍 1. EDA (Electrodermal Activity)
- EDA = sweat gland activity
- Large peaks = phasic responses (rapid skin conductance spikes)
- Slow drifts = tonic levels (overall arousal magnitude)

### 🩸 2. BVP (Blood Volume Pulse)
- Blood volume changes with each hearbeat
- Amplitude variation and noise fluctuation correspond to movement and stress

### ❤️ 3. HR (Heart Rate)
- Elevated periods of HR = stress/physical activity

### 🧭 4. ACC (Accelerometer Magnitude)
- High ACC activity is useful for filtering out motion artifacts in BVP/EDA.
- Movement spikes may correspond to:
    - stress responses
    - transitions between conditions
    - physical movements affecting other sensors
- ACC helps determine which segments are clean and which may be noise-affected.


In [12]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

In [ ]:
base_path = "../data/WESAD/raw/WESAD/S2/S2_E4_Data/"

In [36]:
def load_e4_signal(path):
    """Loads E4 CSV: skip metadata rows and return (values, index, fs)."""
    meta = pd.read_csv(path, header=None, nrows=2)
    start_unix = float(meta.iloc[0, 0])
    fs = float(meta.iloc[1, 0])
    data = pd.read_csv(path, header=None, skiprows=2)[0].values

    n = len(data)
    t_rel = np.arange(n) / fs
    t_abs = start_unix + t_rel
    index = pd.to_datetime(t_abs, unit="s", utc=True)
    return data, index, fs

# EDA
eda_data, eda_index, eda_fs = load_e4_signal(base_path + "EDA.csv")
eda_df = pd.DataFrame({"eda": eda_data}, index=eda_index)
eda_df.index.name = "time"

# HR
hr_data, hr_index, hr_fs = load_e4_signal(base_path + "HR.csv")
hr_series = pd.Series(hr_data, index=hr_index).sort_index()

# TEMP
temp_data, temp_index, temp_fs = load_e4_signal(base_path + "TEMP.csv")
temp_series = pd.Series(temp_data, index=temp_index).sort_index()

# BVP (if you want it later)
bvp_data, bvp_index, bvp_fs = load_e4_signal(base_path + "BVP.csv")

# ACC: 3-axis
acc_meta = pd.read_csv(base_path + "ACC.csv", header=None, nrows=2)
acc_start_unix = float(acc_meta.iloc[0, 0])
acc_fs = float(acc_meta.iloc[1, 0])
acc_raw = pd.read_csv(base_path + "ACC.csv", header=None, skiprows=2).values

t_rel_acc = np.arange(len(acc_raw)) / acc_fs
t_abs_acc = acc_start_unix + t_rel_acc
acc_index = pd.to_datetime(t_abs_acc, unit="s", utc=True)

acc_df = pd.DataFrame(acc_raw, index=acc_index, columns=["acc_x","acc_y","acc_z"])

# IBI (different format)
ibi_df = pd.read_csv(base_path + "IBI.csv", header=None, names=["t_rel", "ibi_sec"])
ibi_df["t_abs"] = eda_index[0].value/1e9 + ibi_df["t_rel"]
ibi_df["time"] = pd.to_datetime(ibi_df["t_abs"], unit="s", utc=True)
ibi_df.set_index("time", inplace=True)

print("EDA:", eda_df.shape, "HR:", hr_series.shape, "TEMP:", temp_series.shape, "ACC:", acc_df.shape, "IBI:", ibi_df.shape)


EDA: (31494, 1) HR: (7865,) TEMP: (31496,) ACC: (251970, 3) IBI: (3459, 3)


In [54]:
# Resample everything onto a common time axis (e.g., 4 Hz)
target_fs = 4  # Hz

common_index = pd.date_range(
    start=eda_df.index.min(),
    end=eda_df.index.max(),
    freq=f"{int(1000/target_fs)}ms"  # milliseconds per sample
)

eda_r = eda_df["eda"].reindex(common_index).interpolate()

hr_r = hr_series.reindex(common_index).interpolate()
temp_r = temp_series.reindex(common_index).interpolate()

# accelerometer magnitude
acc_df = acc_df / 64.0
acc_mag = np.linalg.norm(acc_df[["acc_x","acc_y","acc_z"]].values, axis=1)
acc_mag = pd.Series(acc_mag, index=acc_df.index)
acc_r = acc_mag.reindex(common_index).interpolate()

df = pd.DataFrame({
    "eda": eda_r,
    "hr": hr_r,
    "temp": temp_r,
    "acc": acc_r
}, index=common_index)
df.index.name = "time"

df.describe()


,eda,hr,temp,acc
count,31494.000000,31454.000000,31494.000000,31494.000000
mean,0.547822,75.093992,34.614144,0.990199
std,0.460821,10.735224,4.046567,0.081924
min,0.000000,54.570000,32.310000,0.185537
25%,0.187341,69.190000,33.910000,0.976406
50%,0.334694,73.108750,34.610000,0.984623
75%,0.931864,77.780000,35.490000,0.995105
max,2.451335,134.770000,382.180000,2.537463


In [59]:
def butter_filter(x, fs, low=None, high=None, order=3):
    ny = 0.5 * fs
    if low is not None and high is not None:
        b, a = butter(order, [low/ny, high/ny], btype="band")
    elif low is not None:
        b, a = butter(order, low/ny, btype="high")
    elif high is not None:
        b, a = butter(order, high/ny, btype="low")
    else:
        return x
    return filtfilt(b, a, x)

def eda_features_from_array(x, fs, prom=0.05):
    tonic = butter_filter(x, fs, high=0.05)
    phasic = x - tonic
    peaks, props = find_peaks(phasic, prominence=prom)
    peak_amps = phasic[peaks] if peaks.size else np.array([])
    return {
        "eda_mean": np.nanmean(x),
        "eda_std": np.nanstd(x),
        "eda_tonic_median": np.median(tonic),
        "eda_phasic_mean": np.nanmean(phasic),
        "eda_peak_count": int(peaks.size),
        "eda_peak_amp_mean": float(np.nanmean(peak_amps)) if peak_amps.size else np.nan
    }

def hr_features_from_array(hr_arr):
    hr_arr = np.asarray(hr_arr)
    if hr_arr.size == 0:
        return {"hr_mean": np.nan, "hr_std": np.nan, "hr_na_ratio": np.nan}
    return {
        "hr_mean": float(np.nanmean(hr_arr)),
        "hr_std": float(np.nanstd(hr_arr)),
        "hr_na_ratio": float(np.isnan(hr_arr).mean())
    }

def ibi_hrv_features(ibi_sec):
    ibi = np.asarray(ibi_sec)
    if ibi.size < 2:
        return {"rmssd": np.nan, "sdnn": np.nan, "mean_hr_from_ibi": np.nan}
    diffs = np.diff(ibi)
    rmssd = np.sqrt(np.mean(diffs**2))
    sdnn = np.std(ibi)
    mean_hr = 60.0 / np.mean(ibi)
    return {"rmssd": float(rmssd), "sdnn": float(sdnn), "mean_hr_from_ibi": float(mean_hr)}

def acc_features_from_array(acc_arr):
    arr = np.asarray(acc_arr, dtype=float)
    if arr.ndim == 2 and arr.shape[1] == 3:
        mag = np.linalg.norm(arr, axis=1)
    else:
        mag = arr.flatten()

    # center around baseline gravity (~1g)
    baseline = np.median(mag)
    dyn = np.abs(mag - baseline)  # dynamic component

    # threshold on dynamic part, not on raw magnitude
    movement_mask = dyn > 0.05  # 0.05g deviation from baseline

    return {
        "acc_mean": float(np.nanmean(mag)),
        "acc_std": float(np.nanstd(mag)),
        "acc_energy": float(np.nansum(dyn**2)),   # use dynamic part for energy too
        "acc_pct_movement": float(movement_mask.mean())
    }

def temp_features_from_array(temp_arr):
    x = np.asarray(temp_arr)
    if x.size == 0:
        return {"temp_mean": np.nan, "temp_slope": np.nan}
    t = np.arange(x.size)
    slope = np.polyfit(t, x, 1)[0]
    return {"temp_mean": float(np.nanmean(x)), "temp_slope": float(slope)}

def features_from_window(window_dict, fs_map=None):
    out = {}
    if "eda" in window_dict:
        eda_x = np.asarray(window_dict["eda"])
        fs = fs_map.get("EDA") if fs_map else None
        out.update(eda_features_from_array(eda_x, fs or 4))
    if "hr" in window_dict:
        out.update(hr_features_from_array(window_dict["hr"]))
    # if "ibi_sec" in window_dict:
    #     out.update(ibi_hrv_features(window_dict["ibi_sec"]))
    if "acc" in window_dict:
        out.update(acc_features_from_array(window_dict["acc"]))
    if "temp" in window_dict:
        out.update(temp_features_from_array(window_dict["temp"]))
    return out


In [61]:
window_sec = 30
overlap = 0.5
step_sec = window_sec * (1 - overlap)

starts = []
t0 = df.index[0]
t_end = df.index[-1]
wstart = t0
while wstart + pd.Timedelta(seconds=window_sec) <= t_end:
    wend = wstart + pd.Timedelta(seconds=window_sec)
    starts.append((wstart, wend))
    wstart = wstart + pd.Timedelta(seconds=step_sec)

len(starts), starts[0], starts[1]


(523,
 (Timestamp('2017-05-22 07:15:25+0000', tz='UTC'),
  Timestamp('2017-05-22 07:15:55+0000', tz='UTC')),
 (Timestamp('2017-05-22 07:15:40+0000', tz='UTC'),
  Timestamp('2017-05-22 07:16:10+0000', tz='UTC')))

In [62]:
features_list = []

for start, end in starts:
    # main window from unified df
    w = df.loc[start:end]

    # safe IBI slice (may be empty, that's okay)
    if not ibi_df.empty:
        mask = (ibi_df.index >= start) & (ibi_df.index <= end)
        ibi_w = ibi_df.loc[mask]
        ibi_arr = ibi_w["ibi_sec"].values if "ibi_sec" in ibi_w else np.array([])
    else:
        ibi_arr = np.array([])

    win = {
        "eda":  w["eda"].values,
        "hr":   w["hr"].values,
        "acc":  w["acc"].values,
        "temp": w["temp"].values,
        #"ibi_sec": ibi_arr,
    }

    feats = features_from_window(win, fs_map={"EDA": target_fs})
    feats["start"] = start
    feats["end"] = end
    features_list.append(feats)

features_df = pd.DataFrame(features_list)
print("windows:", len(features_df), "features shape:", features_df.shape)
features_df.head()


windows: 523 features shape: (523, 17)


,eda_mean,eda_std,eda_tonic_median,eda_phasic_mean,eda_peak_count,eda_peak_amp_mean,hr_mean,hr_std,hr_na_ratio,acc_mean,acc_std,acc_energy,acc_pct_movement,temp_mean,temp_slope,start,end
0,0.482373,0.049584,0.490420,0.029246,2,0.204667,85.676914,6.600797,0.330579,0.981821,0.050897,0.315549,0.033058,46.580579,-0.550508,2017-05-22 07:15:25+00:00,2017-05-22 07:15:55+00:00
1,0.554383,0.149962,0.493591,0.006027,3,0.381916,81.319256,1.903322,0.000000,0.995092,0.128024,2.019984,0.264463,35.066694,-0.000787,2017-05-22 07:15:40+00:00,2017-05-22 07:16:10+00:00
2,0.631755,0.153339,0.643049,0.003338,9,0.186843,81.214298,1.394495,0.000000,1.011927,0.266061,8.682833,0.611570,35.150992,0.003365,2017-05-22 07:15:55+00:00,2017-05-22 07:16:25+00:00
3,0.777905,0.178989,0.767257,0.001693,15,0.086938,83.591322,1.479529,0.000000,1.052124,0.334915,14.047696,0.818182,35.357107,0.003509,2017-05-22 07:16:10+00:00,2017-05-22 07:16:40+00:00
4,1.105101,0.235876,1.143987,-0.010162,15,0.093037,86.014463,1.474511,0.000000,1.086563,0.304614,11.482316,0.867769,35.483471,0.001482,2017-05-22 07:16:25+00:00,2017-05-22 07:16:55+00:00


In [63]:
features_df.describe().T

,count,mean,std,min,25%,50%,75%,max
eda_mean,523.0,0.548109,0.457352,0.088107,0.186685,0.335700,0.956251,1.871129
eda_std,523.0,0.041359,0.051095,0.001055,0.008309,0.019771,0.055921,0.345780
eda_tonic_median,523.0,0.546386,0.455872,0.085487,0.186602,0.332752,0.955244,1.864838
eda_phasic_mean,523.0,0.000150,0.003776,-0.021609,-0.000818,-0.000096,0.000481,0.029246
eda_peak_count,523.0,1.407266,2.504499,0.000000,0.000000,0.000000,2.000000,16.000000
eda_peak_amp_mean,237.0,0.101502,0.106755,0.002898,0.047425,0.073336,0.107162,0.826560
hr_mean,523.0,75.090148,10.558392,54.994876,69.271488,73.277727,77.719339,131.493595
hr_std,523.0,1.340064,1.536936,0.066206,0.426300,0.827964,1.659878,11.004585
hr_na_ratio,523.0,0.000632,0.014455,0.000000,0.000000,0.000000,0.000000,0.330579
acc_mean,523.0,0.990225,0.016564,0.971775,0.982439,0.987006,0.992681,1.116682
